<a href="https://colab.research.google.com/github/balkrishnapdl/Autogen-Framework/blob/main/simpleagent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install AutoGen: Use the following command to install AutoGen from the GitHub repository. You may specify a version if necessary to avoid compatibility issues:


In [13]:
pip install pyautogen[gemini]

Import Required Libraries: In your Python environment (e.g., Google Colab), import the necessary modules and set your API key:

In [22]:
import os
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import chromadb
from PIL import Image
from termcolor import colored

import autogen
from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.img_utils import _to_pil, get_image_data
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.code_utils import DEFAULT_MODEL, UNKNOWN, content_str, execute_code, extract_code, infer_lang

Configure the Model: Define the configuration for using the Gemini model:

In [23]:
from google.colab import userdata
geminiapi=userdata.get('GEMINI_API_KEY')
config_list_gemini = [
     {
        "model": "gemini-pro",
        "api_key": geminiapi,
        "api_type": "google"
    }
]

In [24]:

gemini_config = {
    "seed": 25,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list_gemini,
    "request_timeout": 600,
    "retry_wait_time": 120,
}

Initialize Agents: Create instances of AssistantAgent and UserProxyAgent:


In [25]:
assistant = AssistantAgent(
    name="assistant",
    llm_config=gemini_config,
    max_consecutive_auto_reply=10)

user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False
    }
)

Start a Chat Session: Use the agents to initiate a conversation or perform tasks:

In [26]:
user_proxy.initiate_chat(
    assistant,
    message="""Tell me a joke"""
)

user_proxy (to assistant):

Tell me a joke

--------------------------------------------------------------------------------
assistant (to user_proxy):

```sh
# filename: joke.sh
echo "What do you call a fish with no eyes?"
echo "Fsh!"
```

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is sh)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
What do you call a fish with no eyes?
Fsh!


--------------------------------------------------------------------------------
assistant (to user_proxy):

TERMINATE

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Tell me a joke', 'role': 'assistant', 'name': 'user_proxy'}, {'content': '```sh\n# filename: joke.sh\necho "What do you call a fish with no eyes?"\necho "Fsh!"\n```', 'role': 'user', 'name': 'assistant'}, {'content': 'exitcode: 0 (execution succeeded)\nCode output: \nWhat do you call a fish with no eyes?\nFsh!\n', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'TERMINATE', 'role': 'user', 'name': 'assistant'}], summary='', cost={'usage_including_cached_inference': {'total_cost': 0.0005549999999999999, 'gemini-pro': {'cost': 0.0005549999999999999, 'prompt_tokens': 1014, 'completion_tokens': 32, 'total_tokens': 1046}}, 'usage_excluding_cached_inference': {'total_cost': 0.0005549999999999999, 'gemini-pro': {'cost': 0.0005549999999999999, 'prompt_tokens': 1014, 'completion_tokens': 32, 'total_tokens': 1046}}}, human_input=[])